In [26]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

In [2]:
sio = socketio.Server()
app = Flask(__name__)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)


In [ ]:
def process_image(img):
    return img[10:130:2,::4,:]

@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
        
        decoded = base64.b64decode(image_str)
        image=Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
        
        
        #plt.imshow(image_array)
        #plt.show();
        
        img = process_image(image_array)
        img_batch = np.expand_dims(img,axis=0)
        
        
        steering_angle = float(model.predict(img_batch))
        
        throttle = 0.15
       # steering_angle = 0.0
        if speed <15:
            throttle = 0.5
        if speed > 17:
            throttle = -0.1
            

        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)

        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(5592) wsgi starting up on http://0.0.0.0:4567
(5592) accepted ('127.0.0.1', 62532)
127.0.0.1 - - [27/Mar/2020 13:35:12] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 349.497810


In [34]:

if __name__ == '__main__':
    app = socketio.Middleware(sio, app)
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(5592) wsgi starting up on http://0.0.0.0:4567
(5592) accepted ('127.0.0.1', 62521)
127.0.0.1 - - [27/Mar/2020 13:26:41] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 33.489779
(5592) accepted ('127.0.0.1', 62528)
Exception in thread Thread-3253:
Traceback (most recent call last):
  File "C:\Users\benia\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\benia\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\benia\Anaconda3\lib\site-packages\socketio\server.py", line 651, in _handle_event_internal
    r = server._trigger_event(data[0], namespace, sid, *data[1:])
  File "C:\Users\benia\Anaconda3\lib\site-packages\socketio\server.py", line 680, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "<ipython-input-33-84a1f0983893>", line 22, in telemetry
    steering_angle = float(model.predict(img_batch))
  File "C:\Users\benia\Anaconda3\lib\site-packages\k

In [27]:
model = load_model("model.h5")

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 78, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 58, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 76, 32)        18464     
_________________________________________________________________
activation_2 (Activation)    (None, 56, 76, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 38, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 36, 16)        4624      
_________________________________________________________________
activation_3 (Activation)    (None, 26, 36, 16)       